<a href="https://colab.research.google.com/github/minhvu2105/alpha-evm-dex-bot/blob/main/viXTTS_Demo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🔥🔥🔥**viXTTS Demo**🗣️🗣️🗣️

Demo này giúp bạn chạy viXTTS miễn phí trên Google Colab!
Xem thông tin mô hình tại [đây](https://huggingface.co/capleaf/viXTTS)

Bạn có thể dùng demo này với mục đích:
- ✅Mục đích cá nhân, học tập, nghiên cứu, thử nghiệm

Bạn **KHÔNG** được dùng demo này với mục đích:
- ❌Mục đích trái đạo đức, vi phạm pháp luật Việt Nam
- ❌Tạo ra nội dung gây thù ghét, kỳ thị, bạo lực hoặc nội dung vi phạm bản quyền
- ❌Giả mạo danh tính hoặc gây hiểu nhầm rằng nội dung được tạo ra bởi một cá nhân hoặc tổ chức khác

In [1]:
# Thiết lập môi trường tiếng Việt
!rm /etc/localtime
!ln -s /usr/share/zoneinfo/Asia/Ho_Chi_Minh /etc/localtime
!date

# Cài đặt các thư viện cần thiết
!pip install -q --upgrade pip
!pip install -q numpy==1.23.5  # Tránh lỗi xung đột numpy
!pip install -q unidecode underthesea vinorm==2.0.7 gradio==4.35 deepfilternet==0.5.6

# Clone và cài Coqui-TTS (với XTTS + sửa tiếng Việt)
!rm -rf TTS/
!git clone --branch add-vietnamese-xtts -q https://github.com/thinhlpg/TTS.git TTS
!pip install -q -e TTS

!python -m unidic download
!pip install -q git+https://github.com/thinhlpg/TTS.git@add-vietnamese-xtts

# Tải viXTTS model từ Hugging Face
from huggingface_hub import snapshot_download
snapshot_download(repo_id="thinhlpg/viXTTS", repo_type="model", local_dir="model")

# Thông báo hoàn tất
print("✅ Cài đặt xong rồi! Hãy chạy tiếp cell inference phía dưới nhé!")


 > ✅ Cài đặt hoàn tất, bạn hãy chạy tiếp các bước tiếp theo nhé!


In [3]:
# Param đầu vào
language = "Tiếng Việt"
input_text = "Ngân Cương Tử và Ô Linh phu nhân ..."  # Thay bằng đoạn văn bạn thích
reference_audio = "/content/model/samples/nam-truyen-cam.wav"  # Hoặc tải file giọng riêng
normalize_text = True
verbose = True
output_chunks = False

# Imports cần thiết
import os, string
from datetime import datetime
from unidecode import unidecode
from underthesea import sent_tokenize
import torch, torchaudio
from tqdm import tqdm
from vinorm import TTSnorm
from TTS.tts.configs.xtts_config import XttsConfig
from TTS.tts.models.xtts import Xtts
from IPython.display import Audio, clear_output

# Chuẩn hóa text
def normalize_vn(txt):
    return TTSnorm(txt, unknown=False, lower=False, rule=True)

# Hàm lấy tên file bằng thời gian
def get_file_name(txt, maxc=50):
    fn = unidecode(txt[:maxc].lower().replace(" ", "_"))
    fn = fn.translate(str.maketrans("", "", string.punctuation.replace("_","")))
    return datetime.now().strftime("%m%d%H%M%S") + "_" + fn

# Load model XTTS
def load_model():
    config = XttsConfig()
    config.load_json("model/config.json")
    model = Xtts.init_from_config(config)
    model.load_checkpoint(config,
        checkpoint_path="model/model.pth",
        vocab_path="model/vocab.json",
        use_deepspeed=True
    )
    if torch.cuda.is_available():
        model.cuda()
    return model

# Main inference
print("> Đang nạp mô hình viXTTS...")
model = load_model()
clear_output()
print("✅ Đã nạp model thành công!")

# Chuẩn hóa và chia câu
if normalize_text:
    input_text = normalize_vn(input_text)
segments = sent_tokenize(input_text)

# Compute speaker embedding
gpt_latent, spk_emb = model.get_conditioning_latents(
    audio_path=reference_audio,
    gpt_cond_len=model.config.gpt_cond_len,
    max_ref_length=model.config.max_ref_len,
    sound_norm_refs=model.config.sound_norm_refs
)

# Tạo các chunk audio
wavs = []
for s in tqdm(segments):
    if not s.strip(): continue
    out = model.inference(
        text=s, language="vi",
        gpt_cond_latent=gpt_latent,
        speaker_embedding=spk_emb,
        temperature=0.3, length_penalty=1.0,
        repetition_penalty=10, top_k=30, top_p=0.85
    )
    wavs.append(torch.tensor(out["wav"]))

# Ghép và lưu file output
os.makedirs("output", exist_ok=True)
wav_all = torch.cat(wavs).unsqueeze(0)
out_path = f"output/{get_file_name(input_text)}.wav"
torchaudio.save(out_path, wav_all, 24000)

print("✅ Hoàn thành! File output:", out_path)
Audio(out_path)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.4/40.4 MB 47.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 103.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.8/568.8 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.5/4.5 MB 112.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [gradio]
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-cloud-bigquery 3.34.0 requires packaging>=24.2.0, but you have packaging 23.2 which is incompatible.
transformers 4.53.2 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.
yfinance 0.2.65 requires websockets>=13.0, but you have websockets 11.0.3 which is incompatible.
db-dtypes 1.4.3 requires packaging>=24.2.0, but you have pa

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

README.md: 0.00B [00:00, ?B/s]

LICENSE.txt: 0.00B [00:00, ?B/s]

...

vi_sample.wav:   0%|          | 0.00/793k [00:00<?, ?B/s]

nu-nhan-nha.wav:   0%|          | 0.00/764k [00:00<?, ?B/s]

nu-nhe-nhang.wav:   0%|          | 0.00/793k [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

ModuleNotFoundError: Could not import module 'GPT2PreTrainedModel'. Are this object's requirements defined correctly?

In [ ]:
# @title 🎤 **Tải file âm thanh của bạn lên**
# @markdown Để đạt chất lượng tốt nhất, hãy tham khảo file '/content/model/vi_sample.wav'
# @
import os
import locale
from google.colab import files

denoise = True # @param {type:"boolean"}

# Upload the audio file
uploaded = files.upload()
for filename in uploaded.keys():
    # Convert the audio file to WAV format using ffmpeg
    uploaded_dir = os.path.dirname(filename)
    if denoise:
        !deepFilter "{filename}"
        !ffmpeg -i "{filename.replace('.wav', '_DeepFilterNet3.wav')}" -ac 1 -ar 22050 -vn /content/model/user_sample.wav -y -hide_banner -loglevel error
        os.remove(filename.replace('.wav', '_DeepFilterNet3.wav'))
    else:
        !ffmpeg -i "{filename}" -ac 1 -ar 22050 -vn /content/model/user_sample.wav -y -hide_banner -loglevel error
        os.remove(filename)
    break

from IPython.display import Audio, clear_output
clear_output()
print("> Đã tải file âm thanh lên")
Audio("/content/model/user_sample.wav")

In [ ]:
# @title ⏬ **Lưu kết quả vào Drive**
# @markdown Chạy phần này để lưu kết quả vào Google Drive của bạn
# @markdown `/content/drive/MyDrive/vixtts-output`

from google.colab import drive
drive.mount('/content/drive')

# Save the output folder in "vixtts-output" in Google Drive, without overwriting existing files
!cp -n -r /content/output/* /content/drive/MyDrive/vixtts-output
print("> Đã lưu kết quả vào Google Drive")

In [ ]:
# @title ⚠️ **Dọn kết quả**
# @markdown Chạy phần này để xóa toàn bộ file trong `/content/output`
import shutil
shutil.rmtree('/content/output')
print("Đã xóa toàn bộ file trong /content/output")

In [1]:
# @title 📴 **Tắt Demo**
# @markdown Khi chạy xong thì bạn hãy tắt demo để tiết kiệm GPU nhé!
from google.colab import runtime
runtime.unassign()